<a href="https://colab.research.google.com/github/siddfulzele/DataScience/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [201]:
#LOADING RAW DATA
'''
Raw data 1 Data till EoD Apr 19th
Raw data 2 Data till EoD Apr 26th
Raw data 3 Data till EoD May 9th
Raw data 4 Data Live
'''

raw_data = pd.DataFrame()
for i in range(1,5):
    res_data = requests.get("https://api.covid19india.org/raw_data"+str(i)+".json")
    call_data = res_data.json()
    nl_data = pd.DataFrame(call_data['raw_data'])    
    raw_data = raw_data.append(nl_data)

print(raw_data.shape)

(44085, 22)


In [0]:
#LOADING DATA - National Level :Time series, State-wise stats and Test counts
res_NL = requests.get("https://api.covid19india.org/data.json")
call_NL = res_NL.json()
nl_ts = pd.DataFrame(call_NL['cases_time_series'])
nl_tc = pd.DataFrame(call_NL['tested'])
nl_sw = pd.DataFrame(call_NL['statewise'])

In [203]:
print("Rows and colums of time series : ",nl_ts.shape)
print("Rows and colums of tested data : ",nl_tc.shape)
print("Rows and colums of state wise data : ",nl_sw.shape)

Rows and colums of time series :  (106, 7)
Rows and colums of tested data :  (66, 11)
Rows and colums of state wise data :  (38, 11)


In [204]:
print("colum names of TS : ",list(nl_ts))
print("colum names of TC : ",list(nl_tc))
print("colum names of SW : ",list(nl_sw))

colum names of TS :  ['dailyconfirmed', 'dailydeceased', 'dailyrecovered', 'date', 'totalconfirmed', 'totaldeceased', 'totalrecovered']
colum names of TC :  ['individualstestedperconfirmedcase', 'positivecasesfromsamplesreported', 'samplereportedtoday', 'source', 'testpositivityrate', 'testsconductedbyprivatelabs', 'testsperconfirmedcase', 'totalindividualstested', 'totalpositivecases', 'totalsamplestested', 'updatetimestamp']
colum names of SW :  ['active', 'confirmed', 'deaths', 'deltaconfirmed', 'deltadeaths', 'deltarecovered', 'lastupdatedtime', 'recovered', 'state', 'statecode', 'statenotes']


# **Trends Analysis**

In [0]:
#melt function help us to reshape data
daily = nl_ts.melt(id_vars='date', value_vars=['dailyconfirmed','dailydeceased','dailyrecovered'])
daily.replace('dailyconfirmed','Confirmed',inplace=True)
daily.replace('dailydeceased','Deceased',inplace=True)
daily.replace('dailyrecovered','Recovered',inplace=True)
daily.columns = ['Date','Status','Count']
#px.line(daily, x='Date' , y='Count' , color='Status')

In [206]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x = nl_ts.date , y = nl_ts.dailyconfirmed , name = 'Confirmed')
)
fig.add_trace(
    go.Bar(x = nl_ts.date , y = nl_ts.dailydeceased , name = 'Deceased')
)
fig.add_trace(
    go.Bar(x = nl_ts.date , y = nl_ts.dailyrecovered , name = 'Recovered')
)
fig.update_layout(
    title="Spread Trends - Daily ",
    xaxis_title="Date",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)
fig.show()

In [207]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x = nl_ts.date , y = nl_ts.totalconfirmed , name = 'Confirmed')
)
fig.add_trace(
    go.Scatter(x = nl_ts.date , y = nl_ts.totaldeceased , name = 'Deceased')
)
fig.add_trace(
    go.Scatter(x = nl_ts.date , y = nl_ts.totalrecovered , name = 'Recovered')
)
fig.update_layout(
    title="Spread Trends - Cumulative",
    xaxis_title="Date",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)
fig.show()


# **Mortality Rate**

In [182]:
#MORTALITY RATE 
mr = nl_ts.tail(1).iloc[0]
print("Type : ",type(mr))
ta = int(mr['totalconfirmed']) - (int(mr['totaldeceased']) + int(mr["totalrecovered"]))
mr = mr.append(pd.Series({"totalActive" : ta}))
print("Last Updated : ",mr.date)

Type :  <class 'pandas.core.series.Series'>
Last Updated :  14 May 


In [10]:
fig = go.Figure(go.Pie(
    values = [mr.totalrecovered,mr.totaldeceased,mr.totalActive],
    labels = ['Recovered','Death','Active'],
    texttemplate = "%{label}: %{value} <br>(%{percent})",
    textposition = "inside",
    pull=[0, 0.2, 0])
)

fig.update_traces(
    marker=dict(colors=['LightGreen','FireBrick','DeepSkyBlue'])
)

fig.update_layout(
    height = 800,        
    title="MORTALITY RATE ",
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)

fig.show()

# **State Wise Comparison**

In [0]:
nl_sw = nl_sw.drop([0])

In [12]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x = nl_sw.statecode , y = nl_sw.confirmed , name = 'Confirmed')
)
fig.add_trace(
    go.Bar(x = nl_sw.statecode , y = nl_sw.deaths , name = 'Deceased')
)
fig.add_trace(
    go.Bar(x = nl_sw.statecode , y = nl_sw.recovered , name = 'Recovered')
)
fig.update_layout(
    title="State Comparison - Comulative",
    xaxis_title="State Code",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)
fig.show()

# **Testing Process**

In [13]:
list(nl_tc)

['individualstestedperconfirmedcase',
 'positivecasesfromsamplesreported',
 'samplereportedtoday',
 'source',
 'testpositivityrate',
 'testsconductedbyprivatelabs',
 'testsperconfirmedcase',
 'totalindividualstested',
 'totalpositivecases',
 'totalsamplestested',
 'updatetimestamp']

In [14]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x = nl_tc.updatetimestamp , y = nl_tc.totalsamplestested , name = 'Tested')
)

fig.update_layout(
    title="Teste - Daily",
    xaxis_title="Date",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)
fig.show()

# **State Wise Daily Count**

In [0]:
res_SD = requests.get("https://api.covid19india.org/states_daily.json")
call_SD = res_SD.json()
sd_data = pd.DataFrame(call_SD['states_daily'])

In [0]:
sd_data_c = sd_data[sd_data.status == "Confirmed"]

In [17]:
sd_data_c.tail()

,an,ap,ar,as,br,ch,ct,date,dd,dl,dn,ga,gj,hp,hr,jh,jk,ka,kl,la,ld,mh,ml,mn,mp,mz,nl,or,pb,py,rj,sk,status,tg,tn,tr,tt,up,ut,wb
168,0,43,0,2,32,23,0,09-May-20,0,224,0,0,394,2,28,2,13,41,2,0,0,1165,1,0,116,0,0,82,31,0,129,0,Confirmed,31,526,17,3175,159,4,108
171,0,50,0,1,96,4,0,10-May-20,0,381,0,0,398,6,28,1,25,54,7,0,0,1943,0,0,157,0,0,25,61,2,106,0,Confirmed,33,669,16,4311,94,1,153
174,0,38,0,2,42,8,0,11-May-20,0,310,0,0,347,1,27,4,18,14,7,0,0,1230,0,0,171,0,0,38,54,0,174,0,Confirmed,79,798,0,3592,106,0,124
177,0,71,0,0,130,6,0,12-May-20,0,406,0,0,362,8,50,11,55,63,5,0,0,1026,0,0,201,0,0,22,37,0,138,0,Confirmed,51,716,2,3562,91,1,110
180,0,48,0,15,74,4,0,13-May-20,0,359,0,0,364,1,13,5,37,34,10,1,0,1495,0,0,187,0,0,101,10,1,202,0,Confirmed,41,509,1,3726,94,3,117


In [0]:
sd_data_c = sd_data_c.drop(['status','tt'],axis=1)

In [0]:
y = sd_data_c.iloc[:,7].tolist()
sd_data_c = sd_data_c.drop('date',axis=1)

In [0]:
x = list(sd_data_c)

In [0]:
z = sd_data_c.values

In [22]:
fig = go.Figure(go.Surface(
    contours = {
        "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
        "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
    },
    x = x,
    y = y,
    z = z))
fig.update_layout(
        scene = {
            "xaxis": {"nticks": 20},
            "zaxis": {"nticks": 4},
            'camera_eye': {"x": 0, "y": -1, "z": 0.5},
            "aspectratio": {"x": 1, "y": 1, "z": 0.2}
        },
        title="State Wise Daily Count",
        font=dict(
            family="Courier New, monospace",
            size=14,
        )
)
fig.show()

# **Patients by Age**

In [24]:
list(raw_data)

['agebracket',
 'backupnotes',
 'contractedfromwhichpatientsuspected',
 'currentstatus',
 'dateannounced',
 'detectedcity',
 'detecteddistrict',
 'detectedstate',
 'estimatedonsetdate',
 'gender',
 'nationality',
 'notes',
 'numcases',
 'patientnumber',
 'source1',
 'source2',
 'source3',
 'statecode',
 'statepatientnumber',
 'statuschangedate',
 'typeoftransmission',
 'entryid']

In [144]:
age =  raw_data.agebracket[raw_data.agebracket != ""]
age.isnull().sum()

0

In [0]:
age = age[age != '28-35'] #UNWANTED DATA

In [0]:
age = age.astype(float) #COVERT VALUES TO FLOAT


In [0]:
age_data = pd.DataFrame()
age_data['Age'] = age

In [148]:
bins = [x for x in range(0,101,10)]
print(bins)
g_labels = ['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100']
print(g_labels)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']


In [0]:
age_data['group'] = pd.cut(age,bins,labels=g_labels)

In [0]:
age_data = age_data[age_data.Age != 0]

In [151]:
age_data.head()

,Age,group
0,20.0,11-20
3,45.0,41-50
4,24.0,21-30
5,69.0,61-70
6,55.0,51-60


In [152]:
plt_age = age_data.Age.groupby(age_data.group).count()
plt_age

group
0-10       559
11-20     1204
21-30     2833
31-40     2634
41-50     1906
51-60     1291
61-70      636
71-80      204
81-90       46
91-100       4
Name: Age, dtype: int64

In [154]:
smp_size = age_data.Age.count()
print(smp_size)

11317


In [157]:
age_fig = go.Figure()
age_fig.add_trace(
    go.Bar(x=plt_age.index, y = plt_age, name = 'Age')
)
age_fig.update_layout(
    title="Age Comparison - Sample Size "+str(smp_size),
    xaxis_title="Age",
    yaxis_title="Cases",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)
age_fig.show()

# **Gender Comparison**

In [189]:
gen = raw_data[['gender']]
gen.count()

gender    44085
dtype: int64

In [190]:
gen = gen[gen != '']
gen = gen[gen != 'NA']
gen.count()

gender    13753
dtype: int64

In [193]:
type(gen)

pandas.core.frame.DataFrame

In [0]:
gen = gen.groupby(['gender']).size().reset_index(name='counts')

In [195]:
gen

,gender,counts
0,F,4604
1,M,9147
2,Non-Binary,2


In [199]:
gen_fig = go.Figure(go.Pie(
    values = gen.counts,
    labels = ['Females','Males'],
    texttemplate = "%{label}: %{value} <br>(%{percent})",
    textposition = "inside",
    pull=[0, 0])
)

gen_fig.update_traces(
    marker=dict(colors=['LightGreen','FireBrick','DeepSkyBlue'])
)

gen_fig.update_layout(
    height = 600,        
    title="AGE COMPARISON - SAMPLE SIZE "+str(gen.counts.sum()),
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)

gen_fig.show()